## 2. ML Section

In [2]:
from pyspark.sql.functions import regexp_replace, to_timestamp, date_format, col, year, desc, asc, hour, when, sum
#Reading The data
F = "/FileStore/tables/Final P./train_*"
df = spark.read.csv(F,header=True,inferSchema=True)
df = df.withColumn("Dates",to_timestamp(df.Dates, format= "MM/dd/yyyy HH:mm"))
display(df.limit(3))

Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
2015-05-13T23:53:00.000+0000,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425891675136,37.7745985956747
2015-05-13T23:53:00.000+0000,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425891675136,37.7745985956747
2015-05-13T23:33:00.000+0000,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.42436302145,37.8004143219856


#### `` Categorizing String Data: ``

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in list(set(df.columns)-set(['Dates','X','Y'])) ]

pipeline = Pipeline(stages=indexers)
xdf = pipeline.fit(df).transform(df)

xdf = xdf.selectExpr('Dates','Category_index as Category','Descript_index as Descript','DayOfWeek_index as DayOfWeek','PdDistrict_index as\
                      PdDistrict','Resolution_index as Resolution','Address_index as Address','X','Y')
xdf.show(5)

+-------------------+--------+--------+---------+----------+----------+-------+-------------------+------------------+
 Dates|Category|Descript|DayOfWeek|PdDistrict|Resolution|Address| X| Y|
+-------------------+--------+--------+---------+----------+----------+-------+-------------------+------------------+
2015-05-13 23:53:00| 7.0| 5.0| 1.0| 2.0| 1.0| 3991.0| -122.425891675136| 37.7745985956747|
2015-05-13 23:53:00| 1.0| 44.0| 1.0| 2.0| 1.0| 3991.0| -122.425891675136| 37.7745985956747|
2015-05-13 23:33:00| 1.0| 44.0| 1.0| 2.0| 1.0| 7781.0| -122.42436302145| 37.8004143219856|
2015-05-13 23:30:00| 0.0| 0.0| 1.0| 2.0| 0.0| 931.0|-122.42699532676599| 37.80087263276921|
2015-05-13 23:30:00| 0.0| 0.0| 1.0| 8.0| 0.0| 3372.0| -122.438737622757|37.771541172057795|
+-------------------+--------+--------+---------+----------+----------+-------+-------------------+------------------+
only showing top 5 rows

#### `` Creating Feature vector and Normalizing values: ``

In [6]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, MaxAbsScaler

inputCols = list(set(xdf.columns)-set(['Dates','Category']))
assembler = VectorAssembler().setInputCols(inputCols).setOutputCol("features")
transformVector = assembler.transform(xdf)

scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(transformVector)
scaledData = scalerModel.transform(transformVector)
scaledData = scaledData.drop('features').withColumnRenamed('scaledFeatures','features').withColumnRenamed('Category','label')
train, test = scaledData.randomSplit([0.7, 0.3])
scaledData.limit(10).show()

+-------------------+-----+--------+---------+----------+----------+-------+-------------------+------------------+--------------------+
 Dates|label|Descript|DayOfWeek|PdDistrict|Resolution|Address| X| Y| features|
+-------------------+-----+--------+---------+----------+----------+-------+-------------------+------------------+--------------------+
2015-05-13 23:53:00| 7.0| 5.0| 1.0| 2.0| 1.0| 3991.0| -122.425891675136| 37.7745985956747|[0.0625,0.0012759...|
2015-05-13 23:53:00| 1.0| 44.0| 1.0| 2.0| 1.0| 3991.0| -122.425891675136| 37.7745985956747|[0.0625,0.0012759...|
2015-05-13 23:33:00| 1.0| 44.0| 1.0| 2.0| 1.0| 7781.0| -122.42436302145| 37.8004143219856|[0.0625,0.0017695...|
2015-05-13 23:30:00| 0.0| 0.0| 1.0| 2.0| 0.0| 931.0|-122.42699532676599| 37.80087263276921|[0.0,0.0017783484...|
2015-05-13 23:30:00| 0.0| 0.0| 1.0| 8.0| 0.0| 3372.0| -122.438737622757|37.771541172057795|[0.0,0.0012174329...|
2015-05-13 23:30:00| 0.0| 53.0| 1.0| 6.0| 0.0| 2489.0|-122.40325236121201| 37.713430704116|[0.0,1.0616669583...|
2015-05-13 23:30:00| 5.0| 3.0| 1.0| 6.0| 0.0|23142.0| -122.423326976668| 37.7251380403778|[0.0,3.3005006570...|
2015-05-13 23:30:00| 5.0| 3.0| 1.0| 3.0| 0.0|17481.0| -122.371274317441| 37.7275640719518|[0.0,3.7644389193...|
2015-05-13 23:00:00| 0.0| 0.0| 1.0| 9.0| 0.0| 8304.0| -122.508194031117|37.776601260681204|[0.0,0.0013141987...|
2015-05-13 23:00:00| 0.0| 0.0| 1.0| 4.0| 0.0| 6049.0| -122.419087676747| 37.8078015516515|[0.0,0.0019108524...|
+-------------------+-----+--------+---------+----------+----------+-------+-------------------+------------------+--------------------+

####        `#`  `` Classfiers: ``

In [8]:
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


####        `#1`  `` logistic regression ``

In [10]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()

lrModel = lr.fit(train)
predictions = lrModel.transform(test)

predictions.select("prediction", "label", "features").show(5)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

accuracy

+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 0.0| 0.0|[0.04499850832920...|
 0.0| 7.0|[0.04624786179406...|
 1.0| 7.0|[0.04624786179406...|
 0.0| 0.0|[0.02355800826683...|
 2.0| 2.0|[0.02881654281117...|
+----------+-----+--------------------+
only showing top 5 rows

 Out[ 6 ]: 0.25970100450926564

####        `#2`  `` Decision tree ``

In [12]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier()

dtModel = dt.fit(train)
predictions = dtModel.transform(test)

predictions.select("prediction", "label", "features").show(5)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
accuracy

+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 0.0| 0.0|[0.04499850832920...|
 1.0| 7.0|[0.04624786179406...|
 1.0| 7.0|[0.04624786179406...|
 0.0| 0.0|[0.02355800826683...|
 1.0| 2.0|[0.02881654281117...|
+----------+-----+--------------------+
only showing top 5 rows

 Out[ 7 ]: 0.38335221699225136

####        `#3`  `` Random forest ``

In [14]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier()

rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.select("prediction", "label", "features").show(5)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
accuracy

+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 0.0| 0.0|[0.04499850832920...|
 1.0| 7.0|[0.04624786179406...|
 1.0| 7.0|[0.04624786179406...|
 0.0| 0.0|[0.02355800826683...|
 1.0| 2.0|[0.02881654281117...|
+----------+-----+--------------------+
only showing top 5 rows

 Out[ 8 ]: 0.43263476468933215

####        `#4`  `` Naive Bayes ``

In [16]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes()

nbModel = nb.fit(train)
predictions = nbModel.transform(test)
predictions.select("prediction", "label", "features").show(5)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
accuracy

+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 0.0| 0.0|[0.04499850832920...|
 0.0| 7.0|[0.04624786179406...|
 0.0| 7.0|[0.04624786179406...|
 0.0| 0.0|[0.02355800826683...|
 0.0| 2.0|[0.02881654281117...|
+----------+-----+--------------------+
only showing top 5 rows

 Out[ 9 ]: 0.20225539324902858

####        `#5`  `` One-vs-Rest (LogisticRegression) ``

In [18]:
from pyspark.ml.classification import LogisticRegression, OneVsRest

lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

ovr = OneVsRest( classifier=lr )

ovrModel = ovr.fit(train)
predictions = ovrModel.transform(test)
predictions.select("prediction", "label", "features").show(5)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
accuracy

+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 0.0| 0.0|[0.04499850832920...|
 0.0| 7.0|[0.04624786179406...|
 1.0| 7.0|[0.04624786179406...|
 0.0| 0.0|[0.02355800826683...|
 2.0| 2.0|[0.02881654281117...|
+----------+-----+--------------------+
only showing top 5 rows

 Out[ 10 ]: 0.25905845316218906

#### `` Paramter Tuning (Best 3 Classfiers): ``
##### `` #1 logistic regression: ``

In [20]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier

In [21]:

lr = LogisticRegression()

paramGrid = ParamGridBuilder()\
    .addGrid(lr.elasticNetParam,[0.0, 0.5, 1.0])\
    .addGrid(lr.regParam,[0.01, 0.5, 2.0])\
    .addGrid(lr.tol, [1e-04, 1e-06, 1e-08])\
    .build()

pipeline = Pipeline(stages=[lr])
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator())

lrModel = crossval.fit(train)
predictions = lrModel.transform(test)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

bestPipeline = lrModel.bestModel
bestLRModel = bestPipeline.stages[0]
bestParams = bestLRModel.extractParamMap()

accuracy

Out[ 34 ]: 0.2540196350662046

In [22]:
bestParams

Out[ 35 ]: 
{Param(parent='LogisticRegression_bf178aad5b63', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2,
 Param(parent='LogisticRegression_bf178aad5b63', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.0,
 Param(parent='LogisticRegression_bf178aad5b63', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial.'): 'auto',
 Param(parent='LogisticRegression_bf178aad5b63', name='probabilityCol', doc='Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities'): 'probability',
 Param(parent='LogisticRegression_bf178aad5b63', name='threshold', doc='threshold in binary classification prediction, in range [0, 1]'): 0.5,
 Param(parent='LogisticRegression_bf178aad5b63', name='rawPredictionCol', doc='raw prediction (a.k.a. confidence) column name'): 'rawPrediction',
 Param(parent='LogisticRegression_bf178aad5b63', name='labelCol', doc='label column name'): 'label',
 Param(parent='LogisticRegression_bf178aad5b63', name='regParam', doc='regularization parameter (>= 0)'): 0.01,
 Param(parent='LogisticRegression_bf178aad5b63', name='maxIter', doc='maximum number of iterations (>= 0)'): 100,
 Param(parent='LogisticRegression_bf178aad5b63', name='featuresCol', doc='features column name'): 'features',
 Param(parent='LogisticRegression_bf178aad5b63', name='standardization', doc='whether to standardize the training features before fitting the model'): True,
 Param(parent='LogisticRegression_bf178aad5b63', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='LogisticRegression_bf178aad5b63', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0)'): 1e-06,
 Param(parent='LogisticRegression_bf178aad5b63', name='fitIntercept', doc='whether to fit an intercept term'): True}

###### LR Best Paramters are: [``elasticNetParam:0.0``, ``regParam:0.01``, ``tol:1e-06``]

##### `` #2 Decision tree: ``

In [25]:
dt = DecisionTreeClassifier()

paramGrid = ParamGridBuilder()\
    .addGrid(dt.maxMemoryInMB,[256, 512,1024])\
    .addGrid(dt.checkpointInterval,[5, 10,20])\
    .addGrid(dt.maxBins,[32, 64,128])\
    .addGrid(dt.maxDepth,[5, 10,15])\
    .addGrid(dt.minInstancesPerNode,[2, 4,10])\
    .build()

pipeline = Pipeline(stages=[dt])
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator())

dtModel = crossval.fit(train)
predictions = dtModel.transform(test)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

bestPipeline = dtModel.bestModel
bestDTModel = bestPipeline.stages[0]
bestParams = bestDTModel.extractParamMap()

accuracy

Out[ 6 ]: 0.7522980696973607

In [26]:
bestParams

Out[ 25 ]: 
{Param(parent='DecisionTreeClassifier_94bdf0c11a28', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation.'): 256,
 Param(parent='DecisionTreeClassifier_94bdf0c11a28', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent='DecisionTreeClassifier_94bdf0c11a28', name='rawPredictionCol', doc='raw prediction (a.k.a. confidence) column name'): 'rawPrediction',
 Param(parent='DecisionTreeClassifier_94bdf0c11a28', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 32,
 Param(parent='DecisionTreeClassifier_94bdf0c11a28', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='DecisionTreeClassifier_94bdf0c11a28', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='DecisionTreeClassifier_94bdf0c11a28', name='featuresCol', doc='features column name'): 'features',
 Param(parent='DecisionTreeClassifier_94bdf0c11a28', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini',
 Param(parent='DecisionTreeClassifier_94bdf0c11a28', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='DecisionTreeClassifier_94bdf0c11a28', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.'): 1,
 Param(parent='DecisionTreeClassifier_94bdf0c11a28', name='seed', doc='random seed'): 3496922717701084861,
 Param(parent='DecisionTreeClassifier_94bdf0c11a28', name='labelCol', doc='label column name'): 'label',
 Param(parent='DecisionTreeClassifier_94bdf0c11a28', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): True,
 Param(parent='DecisionTreeClassifier_94bdf0c11a28', name='probabilityCol', doc='Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities'): 'probability'}

###### DT Best Paramters are: [``maxMemoryInMB:256``, ``checkpointInterval:10``, ``maxBins:32``, ``maxDepth:5``, ``minInstancesPerNode:1``]

##### `` #3 Random Forest: ``

In [29]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier()

paramGrid = ParamGridBuilder()\
    .addGrid(rf.numTrees,[10, 25, 50])\
    .addGrid(rf.maxDepth,[4, 8, 10])\
    .addGrid(rf.maxBins,[8, 16, 32]) \
    .build()

pipeline = Pipeline(stages=[rf])
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator())

rfModel = crossval.fit(train)
predictions = rfModel.transform(test)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

bestPipeline = rfModel.bestModel
bestRFModel = bestPipeline.stages[0]
bestParams = bestRFModel.extractParamMap()

accuracy

Out[ 36 ]: 0.5767353754512223

In [30]:
bestParams

Out[ 37 ]: 
{Param(parent='RandomForestClassifier_1594dc221845', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False,
 Param(parent='RandomForestClassifier_1594dc221845', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 32,
 Param(parent='RandomForestClassifier_1594dc221845', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent='RandomForestClassifier_1594dc221845', name='probabilityCol', doc='Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities'): 'probability',
 Param(parent='RandomForestClassifier_1594dc221845', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10,
 Param(parent='RandomForestClassifier_1594dc221845', name='rawPredictionCol', doc='raw prediction (a.k.a. confidence) column name'): 'rawPrediction',
 Param(parent='RandomForestClassifier_1594dc221845', name='featuresCol', doc='features column name'): 'features',
 Param(parent='RandomForestClassifier_1594dc221845', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.'): 1,
 Param(parent='RandomForestClassifier_1594dc221845', name='seed', doc='random seed'): -4110891593024638465,
 Param(parent='RandomForestClassifier_1594dc221845', name='featureSubsetStrategy', doc='The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].'): 'auto',
 Param(parent='RandomForestClassifier_1594dc221845', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='RandomForestClassifier_1594dc221845', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation.'): 256,
 Param(parent='RandomForestClassifier_1594dc221845', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini',
 Param(parent='RandomForestClassifier_1594dc221845', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='RandomForestClassifier_1594dc221845', name='labelCol', doc='label column name'): 'label',
 Param(parent='RandomForestClassifier_1594dc221845', name='numTrees', doc='Number of trees to train (>= 1)'): 50,
 Param(parent='RandomForestClassifier_1594dc221845', name='subsamplingRate', doc='Fraction of the training data used for learning each decision tree, in range (0, 1].'): 1.0}

###### RF Best Paramters are: [``numTrees:50``, ``maxDepth:10``, ``maxBins:32``]

## 3. Testing Section

###### `` Selecting Colums that match the (Test Data) and Transforming it: ``

In [34]:
#Selecting Colums that match the (Test Data)
xxdf = xdf.selectExpr('Dates','Category','Descript','DayOfWeek','PdDistrict','Resolution','Address','X','Y')
inputCols = list(set(xdf.columns)-set(['Dates','Category','Descript','Resolution']))
#Feature Vector
assembler = VectorAssembler().setInputCols(inputCols).setOutputCol("features")
transformVector = assembler.transform(xxdf)
#Normalizing
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(transformVector)
scaledData = scalerModel.transform(transformVector)
train = scaledData.drop('features').withColumnRenamed('scaledFeatures','features').withColumnRenamed('Category','label')
train.limit(5).show()

+-------------------+-----+--------+---------+----------+----------+-------+-------------------+------------------+--------------------+
 Dates|label|Descript|DayOfWeek|PdDistrict|Resolution|Address| X| Y| features|
+-------------------+-----+--------+---------+----------+----------+-------+-------------------+------------------+--------------------+
2015-05-13 23:53:00| 7.0| 5.0| 1.0| 2.0| 1.0| 3991.0| -122.425891675136| 37.7745985956747|[0.04357794799292...|
2015-05-13 23:53:00| 1.0| 44.0| 1.0| 2.0| 1.0| 3991.0| -122.425891675136| 37.7745985956747|[0.04357794799292...|
2015-05-13 23:33:00| 1.0| 44.0| 1.0| 2.0| 1.0| 7781.0| -122.42436302145| 37.8004143219856|[0.04433709665847...|
2015-05-13 23:30:00| 0.0| 0.0| 1.0| 2.0| 0.0| 931.0|-122.42699532676599| 37.80087263276921|[0.04302986069898...|
2015-05-13 23:30:00| 0.0| 0.0| 1.0| 8.0| 0.0| 3372.0| -122.438737622757|37.771541172057795|[0.03719848867946...|
+-------------------+-----+--------+---------+----------+----------+-------+-------------------+------------------+--------------------+

###### `` Test Data and it's Transformations: ``

In [36]:
#Reading The Test data
Ft = "/FileStore/tables/Final P./test_*"
dft = spark.read.csv(Ft,header=True,inferSchema=True)
dft = dft.withColumn("Dates",to_timestamp(dft.Dates, format= "MM/dd/yyyy HH:mm"))
display(dft.limit(3))

Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,2015-05-10T23:59:00.000+0000,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.39958770418998,37.7350510103906
1,2015-05-10T23:51:00.000+0000,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391522893042,37.7324323864471
2,2015-05-10T23:50:00.000+0000,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426001954961,37.7922124386284


In [37]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(dft) for column in list(set(dft.columns)-set(['Dates','X','Y'])) ]

pipeline = Pipeline(stages=indexers)
xdft = pipeline.fit(dft).transform(dft)

xdft = xdft.selectExpr('Id','Dates','DayOfWeek_index as DayOfWeek','PdDistrict_index as PdDistrict','Address_index as Address','X','Y')

inputCols = list(set(xdft.columns)-set(['Id','Dates']))
assembler = VectorAssembler().setInputCols(inputCols).setOutputCol("features")
transformVector = assembler.transform(xdft)

scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(transformVector)
scaledData = scalerModel.transform(transformVector)
test = scaledData.drop('features').withColumnRenamed('scaledFeatures','features')
test.limit(5).show()

+---+-------------------+---------+----------+-------+-------------------+----------------+--------------------+
 Id| Dates|DayOfWeek|PdDistrict|Address| X| Y| features|
+---+-------------------+---------+----------+-------+-------------------+----------------+--------------------+
 0|2015-05-10 23:59:00| 6.0| 3.0| 2638.0|-122.39958770418998|37.7350510103906|[0.33333333333333...|
 1|2015-05-10 23:51:00| 6.0| 3.0| 675.0| -122.391522893042|37.7324323864471|[0.33333333333333...|
 2|2015-05-10 23:50:00| 6.0| 2.0| 8645.0| -122.426001954961|37.7922124386284|[0.22222222222222...|
 3|2015-05-10 23:45:00| 6.0| 6.0| 219.0| -122.437393972517|37.7214120621391|[0.66666666666666...|
 4|2015-05-10 23:45:00| 6.0| 6.0| 219.0| -122.437393972517|37.7214120621391|[0.66666666666666...|
+---+-------------------+---------+----------+-------+-------------------+----------------+--------------------+

###### `` Testing : ``

In [39]:
dt = DecisionTreeClassifier(maxMemoryInMB=256, checkpointInterval=5, maxBins=64, maxDepth=15, minInstancesPerNode=2)

Model = dt.fit(train)
predictions = Model.transform(test)


###### `` Return String labels : ``

In [41]:
from pyspark.ml.feature import IndexToString
#Getting the List of 'labels' (String)
xindexers = StringIndexer(inputCol='Category', outputCol='label').fit(df)
xxInt = xindexers.transform(df)
Ze = xxInt.select('Category','label').groupBy('Category','label').count().orderBy(asc("label"))
Zee = list(Ze.select('Category').toPandas()['Category'])
#Mapping each StringLabel with IntLabel (returns Strings)
converter = IndexToString(inputCol="prediction", outputCol="label", labels=Zee)
converted = converter.transform(predictions)
converted.select('Id',"label", "probability", "features").show(5)

+---+--------------+--------------------+--------------------+
 Id| label| probability| features|
+---+--------------+--------------------+--------------------+
 0| NON-CRIMINAL|[0.0,0.0,0.5,0.0,...|[0.33333333333333...|
 1| NON-CRIMINAL|[0.0,0.0,0.5,0.0,...|[0.33333333333333...|
 2| LARCENY/THEFT|[0.33333333333333...|[0.22222222222222...|
 3|OTHER OFFENSES|[0.05970149253731...|[0.66666666666666...|
 4|OTHER OFFENSES|[0.05970149253731...|[0.66666666666666...|
+---+--------------+--------------------+--------------------+
only showing top 5 rows

In [42]:
display(converted)